# Data Mining Project
---

Εισαγωγή Βιβλιοθηκών & Φόρτωση

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Φόρτωση δεδομένων
# Ορίζουμε το '?' ως NaN
df = pd.read_csv('household_power_consumption.txt', sep=';', na_values=['?'], low_memory=False)
print(f"Αρχικά missing values:\n{df.isna().sum()}")

Αρχικά missing values:
Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Καθαρισμός & Interpolation (Χωρίς Warnings)

In [35]:
# Επιλέγουμε τις στήλες που πρέπει να είναι αριθμητικές
numeric_cols = df.columns.difference(['Date', 'Time'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Interpolation μόνο στις αριθμητικές στήλες για να αποφύγουμε το FutureWarning
df[numeric_cols] = df[numeric_cols].interpolate(method='linear')
# Αν έχουν μείνει NaN στην αρχή, τα κλείνουμε με ffill/bfill
df.ffill(inplace=True)
df.bfill(inplace=True)

# Δημιουργία Datetime Index και διαγραφή των παλιών στηλών
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

print("Ο καθαρισμός ολοκληρώθηκε. Missing values:", df.isna().sum().sum())


Ο καθαρισμός ολοκληρώθηκε. Missing values: 0


Feature Engineering (Sub_metering_4 & Resampling)

In [36]:

# Υπολογισμός Sub_metering_4 (σε Wh/min) και clip στο 0
df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']
df['Sub_metering_4'] = df['Sub_metering_4'].clip(lower=0)

# 1. Υπολογισμός Peak Hour Power (Η μέγιστη ωριαία κατανάλωση μέσα στη μέρα)
hourly_consumption = df['Global_active_power'].resample('h').sum()
peak_hour_series = hourly_consumption.resample('D').max()

# 2. Υπολογισμός Night Usage (22:00 - 06:00)
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
night_usage_series = df[night_mask]['Global_active_power'].resample('D').sum()

# Resampling σε ημερήσιο επίπεδο (Daily)
df_daily = df.resample('D').agg({
    'Global_active_power': 'sum',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum',
    'Sub_metering_4': 'sum'
})

# Μετατροπή συνολικής ισχύος σε Wh
df_daily['Daily_total_power'] = df_daily['Global_active_power'] * (1000 / 60)
df_daily['Peak_hour_power'] = peak_hour_series
df_daily['Night_usage_power'] = night_usage_series

Εμπλουτισμός (Lags, Time Features, One-Hot)

In [37]:
# --- 1. Δημιουργία Lags για ΟΛΕΣ τις σημαντικές μεταβλητές ---
# Για να μην έχουμε data leakage στο Regression, πρέπει να έχουμε ιστορικό
# για κάθε πιθανό feature (Τάση, Ένταση, Sub_meterings, κλπ)
features_to_lag = [
    'Daily_total_power', 'Global_reactive_power', 'Voltage',
    'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
    'Sub_metering_3', 'Sub_metering_4', 'Peak_hour_power', 'Night_usage_power'
]

for col in features_to_lag:
    # Lag 1 ημέρας (τι έγινε χθες)
    df_daily[f'{col}_lag_1'] = df_daily[col].shift(1)
    # Lag 7 ημερών (τι έγινε την ίδια μέρα την προηγούμενη εβδομάδα)
    df_daily[f'{col}_lag_7'] = df_daily[col].shift(7)

# --- 2. Rolling Means (Τάση κατανάλωσης) ---
# Κυρίως μας ενδιαφέρει η τάση στη συνολική ισχύ
df_daily['Power_rolling_mean_3'] = df_daily['Daily_total_power'].shift(1).rolling(window=3).mean()
df_daily['Power_rolling_mean_7'] = df_daily['Daily_total_power'].shift(1).rolling(window=7).mean()

# Αφαίρεση NaN που δημιουργήθηκαν από τα shifts (χάνουμε τις πρώτες 7 μέρες)
df_daily.dropna(inplace=True)

# --- 3. Ημερολογιακά Χαρακτηριστικά ---
df_daily['Day_of_week'] = df_daily.index.dayofweek
df_daily['Is_workday'] = (df_daily['Day_of_week'] < 5).astype(int)
df_daily['Is_weekend'] = (df_daily['Day_of_week'] >= 5).astype(int)
df_daily['Month'] = df_daily.index.month

# --- 4. One-Hot Encoding για Εποχές ---
def get_season(m):
    if m in [12,1,2]: return 'Winter'
    elif m in [3,4,5]: return 'Spring'
    elif m in [6,7,8]: return 'Summer'
    else: return 'Autumn'

df_daily['Season'] = df_daily['Month'].apply(get_season)
df_daily = pd.get_dummies(df_daily, columns=['Season'], prefix='Is')

# --- 5. One-Hot Encoding για Ημέρες & Μήνες ---
# Βοηθάει τα μοντέλα να μάθουν μοτίβα τύπου "Κάθε Δευτέρα" ή "Κάθε Αύγουστο"
df_daily = pd.get_dummies(df_daily, columns=['Day_of_week'], prefix='Day')
df_daily = pd.get_dummies(df_daily, columns=['Month'], prefix='Month')

print("Ο εμπλουτισμός ολοκληρώθηκε. Νέες στήλες:", df_daily.columns.tolist())

Ο εμπλουτισμός ολοκληρώθηκε. Νέες στήλες: ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Sub_metering_4', 'Daily_total_power', 'Peak_hour_power', 'Night_usage_power', 'Daily_total_power_lag_1', 'Daily_total_power_lag_7', 'Global_reactive_power_lag_1', 'Global_reactive_power_lag_7', 'Voltage_lag_1', 'Voltage_lag_7', 'Global_intensity_lag_1', 'Global_intensity_lag_7', 'Sub_metering_1_lag_1', 'Sub_metering_1_lag_7', 'Sub_metering_2_lag_1', 'Sub_metering_2_lag_7', 'Sub_metering_3_lag_1', 'Sub_metering_3_lag_7', 'Sub_metering_4_lag_1', 'Sub_metering_4_lag_7', 'Peak_hour_power_lag_1', 'Peak_hour_power_lag_7', 'Night_usage_power_lag_1', 'Night_usage_power_lag_7', 'Power_rolling_mean_3', 'Power_rolling_mean_7', 'Is_workday', 'Is_weekend', 'Is_Autumn', 'Is_Spring', 'Is_Summer', 'Is_Winter', 'Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', '

In [38]:
import requests

print("Λήψη ιστορικών δεδομένων καιρού για Sceaux, France...")

# 1. Ρυθμίσεις για το Open-Meteo API
# Sceaux Coordinates
latitude = 48.778
longitude = 2.295
start_date = "2006-12-16" # Η αρχή του dataset σου
end_date = "2010-11-26"   # Το τέλος του dataset σου

# Ζητάμε Μέση, Ελάχιστη, Μέγιστη Θερμοκρασία (2m) και Βροχόπτωση
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
    "timezone": "Europe/Paris"
}

# 2. Κλήση API
response = requests.get(url, params=params)
data = response.json()

# 3. Μετατροπή σε DataFrame
weather_df = pd.DataFrame({
    'date': pd.to_datetime(data['daily']['time']),
    'Temp_mean': data['daily']['temperature_2m_mean'],
    'Temp_max': data['daily']['temperature_2m_max'],
    'Temp_min': data['daily']['temperature_2m_min'],
    'Precipitation': data['daily']['precipitation_sum']
})

# Ορίζουμε το index για να γίνει σωστά το merge
weather_df.set_index('date', inplace=True)

print(f"Κατέβηκαν {len(weather_df)} ημέρες καιρού.")
display(weather_df.head())

# 4. ΣΥΓΧΩΝΕΥΣΗ ME TO ΚΥΡΙΟ DATAFRAME (df_daily)
# Προσοχή: Το df_daily πρέπει να έχει DatetimeIndex
# (Αν το index σου έχει ώρα, κάνε: df_daily.index = df_daily.index.normalize())

# Κρατάμε backup
df_daily_backup = df_daily.copy()

# Merge (Left join στο index)
df_daily = df_daily.merge(weather_df, left_index=True, right_index=True, how='left')

# Γεμίζουμε τυχόν κενά (π.χ. αν λείπει 1-2 μέρες) με την προηγούμενη τιμή
df_daily[['Temp_mean', 'Temp_max', 'Temp_min', 'Precipitation']] = df_daily[['Temp_mean', 'Temp_max', 'Temp_min', 'Precipitation']].ffill()

print("\nΟ καιρός προστέθηκε επιτυχώς! Νέες στήλες:")
print(df_daily[['Temp_mean', 'Temp_max', 'Precipitation']].head())

Λήψη ιστορικών δεδομένων καιρού για Sceaux, France...
Κατέβηκαν 1442 ημέρες καιρού.


,Temp_mean,Temp_max,Temp_min,Precipitation
date,,,,
2006-12-16,6.1,8.4,3.3,3.8
2006-12-17,2.9,7.0,-0.2,0.0
2006-12-18,3.1,5.2,1.1,0.9
2006-12-19,1.7,4.2,0.6,0.0
2006-12-20,1.1,4.7,-0.9,0.0



Ο καιρός προστέθηκε επιτυχώς! Νέες στήλες:
            Temp_mean  Temp_max  Precipitation
datetime                                      
2006-12-23        1.6       4.0            0.0
2006-12-24        1.4       2.5            0.0
2006-12-25        1.2       2.6            0.0
2006-12-26        0.5       1.2            0.0
2006-12-27        0.2       1.1            0.0


Έλεγχος Θορύβου (Outliers)

In [39]:
# Έλεγχος με Z-score
z_scores = stats.zscore(df_daily['Daily_total_power'])
outliers_count = len(df_daily[abs(z_scores) > 3])
print(f"Εντοπίστηκαν {outliers_count} ακραίες τιμές (Z-score > 3).")

Εντοπίστηκαν 12 ακραίες τιμές (Z-score > 3).


In [40]:
# Έλεγχος για μέρες με μηδενική κατανάλωση
zero_days = df_daily[df_daily['Daily_total_power'] == 0]
print(f"Μέρες με μηδενική κατανάλωση: {len(zero_days)}")

# Έλεγχος για ακραία Voltage
voltage_issues = df_daily[(df_daily['Voltage'] < 220) | (df_daily['Voltage'] > 260)]
print(f"Μέρες με ασυνήθιστη τάση: {len(voltage_issues)}")

Μέρες με μηδενική κατανάλωση: 0
Μέρες με ασυνήθιστη τάση: 0


Κανονικοποίηση (Scaling) & Target Variable

In [41]:
cols_to_scale = []

# Επιλέγουμε για scaling όλες τις αριθμητικές στήλες ΕΚΤΟΣ από τα one-hot (0/1)
# και τα booleans. Δηλαδή: τις αρχικές μετρήσεις + τα lags + τα rolling means.
for col in df_daily.columns:
    # Αγνοούμε τις one-hot (Is_..., Day_..., Month_...) και τα booleans
    if not (col.startswith('Is_') or col.startswith('Day_') or col.startswith('Month_')):
        cols_to_scale.append(col)

print(f"Στήλες που θα γίνουν Scale ({len(cols_to_scale)}): {cols_to_scale}")

scaler = StandardScaler()
df_daily_scaled = df_daily.copy()

# Εφαρμογή Scaling
df_daily_scaled[cols_to_scale] = scaler.fit_transform(df_daily[cols_to_scale])

# Target Variable για Classification (Υψηλή κατανάλωση)
# Προσοχή: Χρησιμοποιούμε το unscaled df_daily για τον υπολογισμό του μέσου όρου
mean_val = df_daily['Daily_total_power'].mean()
df_daily_scaled['High_Consumption'] = (df_daily['Daily_total_power'] > mean_val).astype(int)

print("\nΣτατιστικά κανονικοποιημένων δεδομένων (πρέπει mean~0, std=1):")
display(df_daily_scaled[cols_to_scale].describe().loc[['mean', 'std']].round(2))
print(f"\nClassification Threshold (High Consumption): > {mean_val:.2f} Wh")

Στήλες που θα γίνουν Scale (37): ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Sub_metering_4', 'Daily_total_power', 'Peak_hour_power', 'Night_usage_power', 'Daily_total_power_lag_1', 'Daily_total_power_lag_7', 'Global_reactive_power_lag_1', 'Global_reactive_power_lag_7', 'Voltage_lag_1', 'Voltage_lag_7', 'Global_intensity_lag_1', 'Global_intensity_lag_7', 'Sub_metering_1_lag_1', 'Sub_metering_1_lag_7', 'Sub_metering_2_lag_1', 'Sub_metering_2_lag_7', 'Sub_metering_3_lag_1', 'Sub_metering_3_lag_7', 'Sub_metering_4_lag_1', 'Sub_metering_4_lag_7', 'Peak_hour_power_lag_1', 'Peak_hour_power_lag_7', 'Night_usage_power_lag_1', 'Night_usage_power_lag_7', 'Power_rolling_mean_3', 'Power_rolling_mean_7', 'Temp_mean', 'Temp_max', 'Temp_min', 'Precipitation']

Στατιστικά κανονικοποιημένων δεδομένων (πρέπει mean~0, std=1):


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Daily_total_power,Peak_hour_power,...,Peak_hour_power_lag_1,Peak_hour_power_lag_7,Night_usage_power_lag_1,Night_usage_power_lag_7,Power_rolling_mean_3,Power_rolling_mean_7,Temp_mean,Temp_max,Temp_min,Precipitation
mean,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,...,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0



Classification Threshold (High Consumption): > 26107.53 Wh


In [42]:
# Αποθήκευση
df_daily_scaled.to_csv('household_power_final_preprocessed.csv', index=True)
print("Η προεπεξεργασία ολοκληρώθηκε! Το αρχείο είναι έτοιμο.")

Η προεπεξεργασία ολοκληρώθηκε! Το αρχείο είναι έτοιμο.
